In [50]:

import pandas as pd       
import openpyxl       
from selenium import webdriver       
from selenium.webdriver.common.keys import Keys       
from selenium.webdriver import ActionChains       
from bs4 import BeautifulSoup       
from tkinter import filedialog, Tk       
import time       
from selenium.webdriver.support.ui import WebDriverWait       
from selenium.webdriver.support import expected_conditions as EC       
from selenium.webdriver.common.by import By  
from selenium import webdriver       
import os       
from tqdm import tqdm       
from selenium.common.exceptions import TimeoutException       
from selenium.webdriver.chrome.options import Options       
import matplotlib.font_manager as fm  
import matplotlib.pyplot as plt  
import datetime       
from selenium.webdriver.chrome.service import Service    
from selenium.common.exceptions import UnexpectedAlertPresentException  


def naver(soup) :      
    try :      
        seller = soup.find("span", attrs={"class": "KasFrJs3SA"}).get_text()      
        uploaded_price = soup.find("span", attrs={'class' :'_1LY7DqCnwR' }).get_text()      
        platform = "스마트스토어"       
    except AttributeError :     
        seller = "확인필요"     
        uploaded_price = "확인필요"     
        platform = "스마트스토어"         
    return seller, uploaded_price, platform     
def coupang(soup) :     
    try :      
        seller = soup.find("a", attrs={"class": "prod-sale-vendor-name"}).get_text()      
        uploaded_price = soup.find("span", attrs={'class' :'total-price' }).get_text()      
        platform = "쿠팡"            
    except AttributeError :     
        seller = "확인필요"     
        uploaded_price = "확인필요"     
        platform = "쿠팡"     
    return seller, uploaded_price, platform       
       
def interpark(soup):           
    time.sleep(5)     
    try:     
        seller = soup.find("div", attrs={"class": "sellerName"}).get_text()     
        uploaded_price = soup.find("span", attrs={'class': 'priceWrap'}).get_text()     
        platform = "인터파크"     
    except AttributeError:     
        seller = "확인필요"     
        uploaded_price = "확인필요"     
        platform = "인터파크"     
    return seller, uploaded_price, platform     
def auction(soup) :     
    try :     
        seller = soup.find("a", attrs={"class": "link__seller sp_vipgroup--before sp_vipgroup--after"}).get_text()       
        uploaded_price = soup.find("strong", attrs={"class" :"price_real" }).get_text()      
        platform = "옥션/G마켓"      
    except AttributeError :     
        seller = "확인필요"     
        uploaded_price = "확인필요"     
        platform = "옥션/G마켓"     
    return seller, uploaded_price, platform    
      
def gmarket(soup) :     
    try:     
        seller = soup.find("a", attrs={"class": "link__seller sp_vipgroup--before sp_vipgroup--after"}).get_text()     
        uploaded_price = soup.find("strong", attrs={'class': 'price_real'}).get_text()     
        platform = "G마켓/옥션"     
    except AttributeError:     
        seller = "확인필요"     
        uploaded_price = "확인필요"     
        platform = "G마켓/옥션"     
    return seller, uploaded_price, platform     
def ssg(soup):     
    try:     
        seller = soup.find("a", attrs={"class": "cdtl_info_tit_link"}).get_text()     
        uploaded_price = soup.find("em", attrs={'class': 'ssg_price'}).get_text()     
        platform = "SSG"     
    except AttributeError:     
        seller = "확인필요"     
        uploaded_price = "확인필요"     
        platform = "SSG닷컴"     
    return seller, uploaded_price, platform     
def elevenst(soup):     
    try:     
        seller = soup.find("h1", attrs={"class": "c_product_store_title"}).find('a').get_text().strip()     
        uploaded_price = soup.find("span", attrs={'class': 'num value'}).get_text()     
        platform = "11번가"     
    except AttributeError:     
        seller = "확인필요"     
        uploaded_price = "확인필요"     
        platform = "11번가"     
    return seller, uploaded_price, platform    
      
def wemakeprice(soup):     
    try:     
        seller = soup.find("span", attrs={"class": "store_name"}).get_text()     
        uploaded_price = soup.find("em", attrs={'class': 'num'}).get_text()     
        platform = "위메프"     
    except AttributeError:     
        seller = "확인필요"     
        uploaded_price = "확인필요"     
        platform = "위메프"     
    return seller, uploaded_price, platform     
def lotte(soup):     
    try:     
        seller = soup.find("div", attrs={"class": "top"}).get_text()     
        uploaded_price = soup.find("span", attrs={'class': 'won'}).get_text().strip()     
        platform = "롯데ON"     
    except AttributeError:     
        seller = "확인필요"     
        uploaded_price = "확인필요"     
        platform = "롯데ON"     
    return seller, uploaded_price, platform     
def unknown(soup):     
    seller = "*확인필요*"     
    uploaded_price = ""     
    platform = ""     
    return seller, uploaded_price, platform     
    
            
    
    
dfs = []
            
# 엑셀 파일 읽어오기       
file_path = filedialog.askopenfilename(title="가격지도 파일", defaultextension=".xlsx")       
df_input = pd.read_excel(file_path, header =0)             
            
options = Options()       
options = webdriver.ChromeOptions()       
driver = webdriver.Chrome('./mnt/c/chromedriver.exe', options=options)       
driver.execute_cdp_cmd("Page.addScriptToEvaluateOnNewDocument", {"source": """ Object.defineProperty(navigator, 'webdriver', { get: () => undefined }) """})       
        
         
for i in tqdm(range(len(df_input))):       
    # 검색할 키워드를 주문번호로 대체하여 url 생성       
    keyword = str(df_input.loc[i, '주문번호'])       
    minimum_price = int(df_input.loc[i, '최소값'])       
    maximum_price = int(df_input.loc[i, '최대값'])   
    filter = str(df_input.loc[i, '필터'])    
                
    #시트에서 지도가 추출하기       
    comsmart_standard = int(df_input.loc[i, '지도가'])       
                
    url = f'https://search.shopping.naver.com/search/all?origQuery={keyword}&pagingIndex=1&pagingSize=80&productSet=total&query={keyword}&sort=rel&timestamp=&viewType=list'     
                
    driver.get(url)       
    time.sleep(2)       
                
    search_btns = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "a[data-testid='SEARCH_SUB_FILTER_ORDER_LIST']")))       
                
    for btn in search_btns:       
        if btn.text.strip() == "낮은 가격순":       
            btn.click()       
            break       
                
    #최적화 필터 제거하기           
    next_link = WebDriverWait(driver, 50).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'a.subFilter_btn_radio__13PEL')))       
    if 'on' in next_link.get_attribute('class'):       
        next_link.click()       
                    
    time.sleep(3)       
                    
            
    min_price = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "//input[@title='최소가격 입력']")))     
    min_price.clear()     
    min_price.send_keys(minimum_price)     
        
    time.sleep(1)     
        
    st_date = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "//input[@title='최대가격 입력']")))     
    st_date.clear()      
    st_date.send_keys(maximum_price)      
        
    time.sleep(1)     
        
    search_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//a[@class='filter_price_srh__D6arg' and @role='button']")))        
    search_button.click()     
        
    time.sleep(1)     
                
    for c in range(0, 20):       
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight/20*" + str(c+1) + ");")       
            time.sleep(0.1)       
                    
    time.sleep(3)    

    des_list = []
    link_list = []
    seller_list = []
    prc_list = []
    gap_list = []
    comment = []
    platform_list = []  
    
    html = driver.page_source       
    soup = BeautifulSoup(html, 'html.parser')        
    flatform = ""
    image_list = soup.select("div.product_item__MDtDF")        
            
    for item in image_list:
        for des in item.select("a.product_link__TrAac.linkAnchor"):
            des_text = des['title']
            des_list.append(des_text)
            link_text = des['href']
            link_list.append(link_text)
    for item in image_list:
        for seller in item.select_one("div.product_mall_title__Xer1m > a"):
            seller_text = seller.get_text()
            seller_list.append(seller_text)

    for item in image_list:
        platform = item.select_one("div.product_mall_title__Xer1m > a")
        if platform.img :
            platform_name = platform.img['alt']
        else :
            platform_name = "네이버"
            
        platform_list.append(platform_name)

    for item in image_list:        
        for price in item.select("span.price_num__S2p_v"):
            prc = int(price.get_text().replace("원", "").replace(",", ""))
            prc_list.append(prc)
            gap = comsmart_standard - prc
            gap_list.append(gap)
            comment_text = "안녕하세요. 컴스마트 관리부입니다. 현재 업로드 하신 제품의 당사 지도가는 " + str(comsmart_standard) + "원으로 현재 업로드하신 금액과는 당사 지도가 대비" + str(gap) + "원 차이가 있으니 판매 단가 수정을 부탁드립니다." 
            comment.append(comment_text)  

    df = pd.DataFrame({  
        "모델명" : [keyword]*len(des_list),               
        "상세정보": des_list,        
        "지도가" : [comsmart_standard]*len(des_list),          
        "업체등록가": prc_list,      
        "가격차이": gap_list,  
        "판매처" : seller_list,  
        "플랫폼" : platform_list,
        "링크주소": link_list,
        "안내문구" : comment,   
        })     
    
    if filter :
        df = df[~df['상세정보'].str.contains(filter)]
    
    dfs.append(df.copy())
df_total = pd.concat(dfs, ignore_index=True)
df_total = df_total[ df_total['판매처'] != '쇼핑몰별 최저가' ]
ddf_total = df_total[df_total['가격차이'] >= 1].reset_index(drop=True)


C:\Users\Owner\AppData\Local\Temp\ipykernel_17036\4108289390.py:135: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('./mnt/c/chromedriver.exe', options=options)
100%|██████████| 6/6 [01:34<00:00, 15.73s/it]


In [51]:
df

,모델명,상세정보,지도가,업체등록가,가격차이,판매처,플랫폼,링크주소,안내문구
0,FW452,FW452 Coms USB C타입 멀티 허브 도킹스테,28050,28050,0,이더텍정보,네이버,https://cr.shopping.naver.com/adcr.nhn?x=eiTpT...,안녕하세요. 컴스마트 관리부입니다. 현재 업로드 하신 제품의 당사 지도가는 2805...
1,FW452,FW452 Coms USB C타입 멀티 허브 도킹스테,28050,28050,0,미우 스토어,네이버,https://cr.shopping.naver.com/adcr.nhn?x=zUo4w...,안녕하세요. 컴스마트 관리부입니다. 현재 업로드 하신 제품의 당사 지도가는 2805...
2,FW452,FW452 USB C타입 멀티 허브 도킹스테이션,28050,28050,0,,인터파크쇼핑,https://cr.shopping.naver.com/adcr.nhn?x=GGTKK...,안녕하세요. 컴스마트 관리부입니다. 현재 업로드 하신 제품의 당사 지도가는 2805...
3,FW452,FW452 Coms USB C타입 멀티 허브 도킹스테,28050,28050,0,림아일체,네이버,https://cr.shopping.naver.com/adcr.nhn?x=ks7ro...,안녕하세요. 컴스마트 관리부입니다. 현재 업로드 하신 제품의 당사 지도가는 2805...
4,FW452,FW452 Coms USB C타입 멀티 허브 도킹스테,28050,28050,0,바룸 스퀘아,네이버,https://cr.shopping.naver.com/adcr.nhn?x=MUx%2...,안녕하세요. 컴스마트 관리부입니다. 현재 업로드 하신 제품의 당사 지도가는 2805...
5,FW452,FW452 Coms USB C타입 멀티 허브 도킹스테,28050,28050,0,일렉트로해피,네이버,https://cr.shopping.naver.com/adcr.nhn?x=OIPnN...,안녕하세요. 컴스마트 관리부입니다. 현재 업로드 하신 제품의 당사 지도가는 2805...
6,FW452,FW452 Coms USB C타입 멀티 허브 도킹스테,28050,28050,0,오사자,네이버,https://cr.shopping.naver.com/adcr.nhn?x=VVAuK...,안녕하세요. 컴스마트 관리부입니다. 현재 업로드 하신 제품의 당사 지도가는 2805...
7,FW452,FW452 Coms USB C타입 멀티 허브 도킹스테,28050,28050,0,VAROOM,네이버,https://cr.shopping.naver.com/adcr.nhn?x=h47h6...,안녕하세요. 컴스마트 관리부입니다. 현재 업로드 하신 제품의 당사 지도가는 2805...
8,FW452,FW452 Coms USB C타입 멀티 허브 도킹스테,28050,28050,0,아이엠멀티샵,네이버,https://cr.shopping.naver.com/adcr.nhn?x=aUaK%...,안녕하세요. 컴스마트 관리부입니다. 현재 업로드 하신 제품의 당사 지도가는 2805...
9,FW452,FW452 Coms USB C타입 멀티 허브 도킹스테,28050,28050,0,비바띵스,네이버,https://cr.shopping.naver.com/adcr.nhn?x=ale%2...,안녕하세요. 컴스마트 관리부입니다. 현재 업로드 하신 제품의 당사 지도가는 2805...


네이버
네이버
인터파크쇼핑
네이버
네이버
네이버
네이버
네이버
네이버
네이버
네이버
네이버
옥션
하이마트쇼핑몰
롯데ON
네이버
네이버
네이버
네이버
G마켓
옥션
네이버
쿠팡
네이버
네이버
네이버
네이버
네이버
네이버
네이버
네이버
네이버
네이버
네이버
네이버
네이버
쿠팡
네이버
네이버
네이버
옥션
네이버
네이버
위메프
네이버
네이버
네이버
네이버
네이버
네이버
